# ANU Inversion #1:  Experimental Sphere Inversion

This notebook handles ANU's Inversion Problem #1.  It involves the detection of worlds built from multiple uniform spherical distributions of mass, with density contrast comparable to the porosity of the medium.  The idea is to tie some hard numbers to sensor specifications.

For the moment, we aren't dealing with uncertainty in the background geology at all.  Thus most of the problems of non-uniqueness associated with uncertain geology should not be problems, and the question will be mainly whether we can pick up and disambiguate signals from multiple spheres.  I have some expectations in advance:

* The significance of a detection for a single source will be proportional to the summed signal-to-noise inside the FWHM (at the surface) of the true gravity signal (related to the depth of a source).  We can probably work out, on paper, how that depends on the source depth and the peak SNR.
* Difficult edge cases will include spheres closer laterally than the effective resolution at the given depth, as well as spheres stacked vertically.

Let's start setting this up, then.

## Back-of-the-envelope estimate for a single source

Let's flesh out our intuitions from above.  The vertical component of gravity for a point source of mass $M$ is

$$g_z = \frac{GMz}{(x^2 + y^2 + z^2)^{3/2}}$$

For a detector directly over the source at $x = y = 0$, the signal attains a peak value of $GM/z^2$.  The two-dimensional signal pattern on the ground also has a full width at half maximum (FWHM) of $\sqrt{2^{2/3} - 1} \times z = 0.766 z$.  Both the mass and depth of the source can be determined uniquely from data, up to noise.

We can naively compare the gravitational source detection problem to point-source photometry in astronomical images.  Suppose the survey pattern is a regular grid of spacing $d$, with a noise variance $\sigma_0$ on each individual gravitational field measurement.  The "point spread function" and hence the resolution of the instrument depends on the depth of the source to which sensitivity is desired.  The depth sets the FWHM $R$ of the point-source response, and by analogy, one detects a source at location $(x, y, z)$ by integrating all the signal within radius $R = 0.766 z$ and comparing it to some desired threshold of significance.

This isn't precisely how Bayesian source detection works, since Bayesian reasoning returns a probability rather than a yes-no answer based on a threshold cut. But this example can give us an idea of how detector performance will scale. Since the Bayesian method accepts $z$ as a fit parameter (as in "PSF-fitting photometry"), the fit should be roughly equivalent to using the optimal value of $R$, as opposed to an "aperture photometry" approach where $R$ is fixed _a priori_.  The Bayesian method should also be able (to a point) to disentangle superimposed signals from vertically stacked masses, as opposed to a pure aperture-integration method which cannot.

More concretely, if the grid spacing $d$ is small compared to $R$ (less than $R/2$ at least, to achieve a Nyquist-sampled signal), we can set $r^2 = x^2 + y^2$ and integrate signal per unit area out to radius $R$:

$$\int_0^R \frac{GMz}{(r^2 + z^2)^{3/2}} \frac{2\pi r}{d^2} \, dr = \frac{2\pi GMz}{d^2} \int_{z^2}^{R^2+z^2} u^{-3/2} \, du
    = \frac{4\pi GM}{d^2} \left( 1 - \frac{z}{\sqrt{R^2 + z^2}} \right)$$

which attains its maximum at $R = 1.25z$.  The integrated noise variance will simply be $(4\pi R^2/d^2) \sigma_0^2$, so that the optimal signal to noise at $R = 1.25z$ is very nearly $GM/dz\sigma_0$.

If we fix a maximum integration time budget $T$ for a given area $A$ and consider that $\sigma_0$ itself depends on integration time e.g. $\sigma_0^2 \propto T$, then the answer also becomes insensitive to the precise grid spacing:  the number of measurements per unit area is $A/d^2 \propto T$, so that the product $d\sigma_0$ is a constant for a given survey time budget.  Thus integration time can be equally spent either on more grid points or on longer integrations, but as long as the signal is Nyquist sampled for sources _at the desired depth_, the result shouldn't matter much.  This also allows us to scale the above detection limit with total integration time to achieve a sensitivity threshold.

In summary, for a survey with a given time budget:
* **The ideal grid spacing will be no greater than $z_\mathrm{min}/2$ where $z_\mathrm{min}$ is the _minimum_ depth beneath the surface of water features to which we want the survey to be sensitive.**  This will most likely be set by some quantile of our prior on the depth of the water table.  If we want to minimize the number of survey points, this suggests that the ideal grid spacing should be _equal_ to $z_\mathrm{min}/2$.
* **The limiting resolution for _lateral variations in groundwater_ at a given depth $z$ will be $\max(d, z/2)$.**  Note that this will _not_ be constant across the survey, and resolution of fine detail will degrade as a function of the actual source depth, which complicates our analogy with astronomical images.
* **The limiting resolution for _lateral variations in soil moisture_ (at $z = 0$) will simply be the survey grid spacing $d$.** Soil moisture variations may therefore be difficult or impossible to disentangle from variations in groundwater at the limiting depth.  Any additional constraints on soil moisture from complementary datasets should help minimize non-uniqueness in groundwater imaging.

We'll see in due course whether the actual experiments bear out this impression.  Additional complexity such as the ability to measure different tensor components may become important once a source has been detected and additional information about its shape is required, but I don't expect these to matter much for source detectability.

## Elements of the statistical model

For now let's just assume that the likelihood is iid Gaussian, which if I understand the detectors correctly should be true to a good approximation (for a given integration time) on these grids.

The prior will be slightly trickier, because the label-switching problem for $N$ spheres means that the inversion will have $N!$ modes, all corresponding to the same interpretation.  To deal with this, let's impose a strict order on the $z$-coordinate of the spheres, so that each set of parameters will have a unique label.  [This paper](https://arxiv.org/abs/1907.11631) details a solution, at least for uniform priors:

> For concreteness, we provide here a pseudo-Python
implementation of Eq. 5. The input x (in C) and output X (in T ) are numpy arrays where all values are in the prior range (0, 1). The values of x may be in any order whilst the values of X are, by construction, in ascending order. If a different prior range is needed then the input and output must be shifted and rescaled as appropriate. A full Python implementation (including the shifting and rescaling) is provided at the GitHub repository [23].

In [ ]:
import numpy as np

def phi(x):
    K = len(x)
    i = numpy.arange(K)
    inner = np.power(1 - x, 1/(K - i))
    X = 1 - np.cumprod(inner)
    return X